In [1]:
# Install required packages
!pip install torch transformers peft datasets sentence-transformers einops triton

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 114.3 MB/s eta 0:00:0000:01:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 KB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 KB 128.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 KB 118.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 199.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 156.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 KB 122.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 KB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.7 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 185.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 190.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 137.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 197.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 235.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 125.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 235.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 213.6 MB/s eta 0:00:00


In [5]:
# Required imports
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments
from datetime import datetime

# Load processed JSON dataset
json_file = 'Processed_History.json'

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert JSON to DataFrame
df = pd.DataFrame(data)
df.head()

,Title,URL,Timestamp,Domain
0,Sign in - Google Accounts,https://accounts.google.com/v3/signin/challeng...,2024-10-04 00:27:37,accounts.google.com
1,Sign in - Google Accounts,https://accounts.google.com/v3/signin/challeng...,2024-10-04 00:27:36,accounts.google.com
2,Home - Google Drive,https://drive.google.com/drive/u/0/home,2024-10-04 00:27:00,drive.google.com
3,Google Drive,https://drive.google.com/drive/u/0/,2024-10-04 00:26:59,drive.google.com
4,Google Takeout,https://takeout.google.com/,2024-10-04 00:26:28,takeout.google.com


In [6]:
# Prepare the data for fine-tuning
def preprocess_row(row):
    input_text = f"Title: {row['Title']} Domain: {row['Domain']} Timestamp: {row['Timestamp']}"
    output_text = f"URL: {row['URL']}"
    return {"input_text": input_text, "output_text": output_text}

formatted_data = df.apply(preprocess_row, axis=1).tolist()
formatted_data[:5]

[{'input_text': 'Title: Sign in - Google Accounts Domain: accounts.google.com Timestamp: 2024-10-04 00:27:37',
  'output_text': 'URL: https://accounts.google.com/v3/signin/challenge/pk?TL=APps6ealSSf4ma1-X-ntHMU9eaTPwQncF_HL0e4znIkCfXmRwXfJufYV72czbVkO&cid=1&continue=https%3A%2F%2Ftakeout.google.com%2Ftakeout%2FtoDrive%3FpreauthUserSessionId%3D0%26profile%3DChEKD2FkZGl0aW9uYWxfZGF0YQoICgZhbGVydHMKCQoHY2hlY2tpbgoSChBhcnRzX2FuZF9jdWx0dXJlCgwKCmNvdXJzZV9raXQKCQoHYmxvZ2dlcgoKCghjYWxlbmRhcgoeChxjaHJvbWVfb3NfZHJhd2luZ19hcHBfY2FudmFzCggKBmNocm9tZQoPCg1jbGFzc2ljX3NpdGVzCgoKCGNvbnRhY3RzCgwKCmNyaXNpc191Z2MKHgocY2hyb21lX29zX2hhbmR3cml0aW5nX2FwcF9hNAoaChhkYXRhX3NoYXJlZF9mb3JfcmVzZWFyY2gKCgoIZGlzY292ZXIKBwoFZHJpdmUKCAoGZW1iYXJrCgUKA2ZpdAoICgZmaXRiaXQKEAoOZ29vZ2xlX2FjY291bnQKDQoLbXlfYnVzaW5lc3MKDwoNaGFuZ291dHNfY2hhdAoVChNnb29nbGVfY2xvdWRfc2VhcmNoChQKEmRldmVsb3Blcl9wbGF0Zm9ybQoHCgVlYXJ0aAoKCghmZWVkYmFjawoQCg5nb29nbGVfZmluYW5jZQoRCg9zdXBwb3J0X2NvbnRlbnQKBgoEbWVldAoMCgpnb29nbGVfb25lCgwKCmdvb2dsZV9wYXkK

In [7]:
# Create a Hugging Face Dataset
dataset = Dataset.from_list(formatted_data)

# Split the dataset into train, validation, and test sets
train_data = dataset.train_test_split(test_size=0.3, seed=42)
val_data, test_data = train_data['test'].train_test_split(test_size=0.5, seed=42).values()

train_dataset = train_data['train']
val_dataset = val_data
test_dataset = test_data

In [8]:
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

114254
24484
24483


In [10]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.0 MB/s eta 0:00:00a 0:00:01


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pretrained model and tokenizer
model_id = "OpenNLPLab/TransNormerLLM-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- configuration_transnormer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- srmsnorm_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenNLPLab/TransNormerLLM-1B:
- utils.py
- norm.py
. Make sure to double-check

In [20]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    target_modules=[
        "token_mixer.qkvu_proj",  # Main attention mechanism
        "channel_mixer.l1",  # Channel transformations
        "channel_mixer.l2",
        "channel_mixer.l3"
    ],
    lora_dropout=0.1,
)

model = get_peft_model(model, lora_config)

2024-12-08 10:12:37 | INFO | peft.tuners.tuners_utils | Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!


In [21]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt",
    pad_to_multiple_of=8  # Aligns tensors to multiples of 8 for better performance on H100
)

In [15]:
# Check if the tokenizer has a pad token, and set it if not
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the datasets
def tokenize_function(examples):
    inputs = tokenizer(examples["input_text"], truncation=True, max_length=512, padding="max_length")
    outputs = tokenizer(examples["output_text"], truncation=True, max_length=512, padding="max_length")
    inputs["labels"] = outputs["input_ids"]
    return inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 24483/24483 [00:10<00:00, 2401.97 examples/s]


In [16]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should display "NVIDIA H100"

True
NVIDIA H100 80GB HBM3


In [17]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [23]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  # Match this with the save strategy
    save_strategy="steps",  # Match this with the eval strategy
    save_steps=500,         # Specify how frequently to save
    eval_steps=500,         # Specify how frequently to evaluate
    learning_rate=3e-5,
    fp16=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model_new")
tokenizer.save_pretrained("./fine_tuned_model_new")

Step,Training Loss,Validation Loss
500,3.376300,0.810791
1000,3.114700,0.750926
1500,2.962500,0.718027
2000,2.829200,0.697403
2500,2.783300,0.684619
3000,2.637900,0.674451
3500,2.598200,0.663509
4000,2.636700,0.655807
4500,2.564800,0.650469
5000,2.640900,0.646504


('./fine_tuned_model_new/tokenizer_config.json',
 './fine_tuned_model_new/special_tokens_map.json',
 './fine_tuned_model_new/tokenizer.model',
 './fine_tuned_model_new/added_tokens.json')

In [26]:
from transformers import AutoModelForCausalLM

base_model_path = "OpenNLPLab/TransNormerLLM-1B"  # Original base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_path, trust_remote_code=True)

In [27]:
from peft import PeftModel

adapter_path = "./fine_tuned_model_new"
model_with_lora = PeftModel.from_pretrained(base_model, adapter_path)

/home/ubuntu/.local/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.token_mixer.qkvu_proj.lora_A.default.weight', 'base_model.model.model.layers.0.token_mixer.qkvu_proj.lora_B.default.weight', 'base_model.model.model.layers.0.channel_mixer.l1.lora_A.default.weight', 'base_model.model.model.layers.0.channel_mixer.l1.lora_B.default.weight', 'base_model.model.model.layers.0.channel_mixer.l2.lora_A.default.weight', 'base_model.model.model.layers.0.channel_mixer.l2.lora_B.default.weight', 'base_model.model.model.layers.0.channel_mixer.l3.lora_A.default.weight', 'base_model.model.model.layers.0.channel_mixer.l3.lora_B.default.weight', 'base_model.model.model.layers.1.token_mixer.qkvu_proj.lora_A.default.weight', 'base_model.model.model.layers.1.token_mixer.qkvu_proj.lora_B.default.weight', 'base_model.model.model.layers.1.channel_mixer.l1.lora_A.default.weight', 'base_model.model.mode

In [28]:
model_with_lora = model_with_lora.merge_and_unload()

In [30]:
model_with_lora.save_pretrained("./merged_fine_tuned_model", safe_serialization=False)
tokenizer.save_pretrained("./merged_fine_tuned_model")

('./merged_fine_tuned_model/tokenizer_config.json',
 './merged_fine_tuned_model/special_tokens_map.json',
 './merged_fine_tuned_model/tokenizer.model',
 './merged_fine_tuned_model/added_tokens.json')

In [31]:
# # Load fine-tuned model for inference
# fine_tuned_model_path = "./fine_tuned_model_new"
# tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path, trust_remote_code=True)

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
model_path = "./merged_fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
from transformers import pipeline

# Create a text-generation pipeline for inference
rag_pipeline = pipeline("text-generation", model=model_with_lora, tokenizer=tokenizer)

# Example query
input_text = "Title: Learn Python programming Domain: youtube.com Timestamp: 2024-12-06 10:00:00"
generated_text = rag_pipeline(input_text, max_length=50)
print(generated_text)

# Optional: Generate embeddings for similarity search
from sentence_transformers import SentenceTransformer

# Load a SentenceTransformer model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for the dataset
df["Embedding"] = df["Title"].apply(lambda x: embedding_model.encode(x).tolist())

# Save embeddings for future use
df.to_csv("chrome_history_with_embeddings.csv", index=False)

print("Fine-tuning and embedding generation completed.")

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=2048) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Title: Learn Python programming Domain: youtube.com Timestamp: 2024-12-06 10:00:00 UTC Timestamp: 2024-12-06 12:00:00 UTC Timestamp: 2025-12-06 10:00:00 UTC Timestamp: 2025-12-06 01:15:00 UTC Timestamp: 2025-12-06 09:00:00 UTC Timestamp: 2026-12-06 08:00:00 UTC Timestamp: 2025-12-07 19:00:00 UTC Timestamp: 2025-12-07 11:00:00 UTC Timestamp: 2026-12-07 11:00:00 UTCTimestamp: 2026-12-09 03:05:00 UTC Timestamp: 2027-12-02 16:00:00 UTC Timestamp: 2026-12-10 05:00:00 UTC Timestamp: 2027-12-11 18:00:00 UTC Timestamp: 2026-12-12 02:40:00 UTC Timestamp: 2026-12-20 02:47:00 UTC Timestamp: 2026-12-22 03:56:00 UTC Timestamp: 2026-12-28 02:03:00 UTC Timestamp: 2026-12-31 00:20:00 UTC Timestamp: 2026-12-10 08:30:00 UTC Timestamp: 2026-12-11 17:14:00 UTC Timestamp: 2026-12-15 18:38:00 UTC Timestamp:\n\nExplanation:\nThe range [1,10] represents all years from 1, to 10, inclusive. \n\nThe set of numbers mentioned in the title of the program is not in the interval mentioned in the 

Batches: 100%|██████████| 1/1 [00:00<00:00, 340.70it/s]
